In [6]:
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files

In [7]:
def Read_image(image_path) :
    image = Image.open(image_path)
    image = image.convert("RGB")
    pixels = image.load()
    return image,pixels

def Show_image(image) :
    plt.imshow(image)
    plt.axis("off")
    plt.show()

In [8]:
def encode_message(image_path, message):
    image,pixels = Read_image(image_path)

    # Convert message to binary and add delimiter
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    binary_message += '1111111111111110'  # Special delimiter to mark end

    width, height = image.size
    index = 0

    for y in range(height):
        for x in range(width):
            if index < len(binary_message):
                r, g, b = pixels[x, y]
                # Modify the least significant bit (LSB) of the red channel
                r = (r & ~1) | int(binary_message[index])
                pixels[x, y] = (r, g, b)
                index += 1

    return image



In [9]:
def start_encode(image_path,message,output_path) :
  encoded_image = encode_message(image_path, message)
  response= 'y'
  if response=='y' :
    # Save in PNG format to prevent compression artifacts
    encoded_image.save(output_path, "PNG")
    print(f"Stego image saved as {output_path}")

    # Download the image using Colab
    files.download(output_path)
  else :
   print("Image not saved")
  return encoded_image

In [ ]:
image_path = "volley image.jpeg"  # Replace with your actual image path
message = "Hello"
output_path= "newencode.png"
encoded_image=start_encode(image_path, message, output_path)
Show_image(encoded_image)

In [10]:
def decode_message(image_path):
    encoded_image = Image.open(image_path).convert("RGB")
    pixels = encoded_image.load()
    width, height = encoded_image.size

    binary_message = ""

    for y in range(height):
        for x in range(width):
            r, _, _ = pixels[x, y]
            binary_message += str(r & 1)

    # Stop at delimiter
    if "1111111111111110" in binary_message:
        binary_message = binary_message.split("1111111111111110")[0]

    chars = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]
    message = ''.join(chr(int(char, 2)) for char in chars)

    return message




In [15]:
# Use the downloaded stego image for decoding
image_to_decode="newencode.png"
decoded_message = decode_message(image_to_decode)
print("Decoded Message:", decoded_message)

Decoded Message: Hello
